In [1]:
import pandas as pd
import numpy as np
import os
import time
import scipy.stats as stats
from utils import load_data, val_loglhood, loglhood

# Gibbs Sampling

In [2]:
def gibbs_sampling(iters, data_path, K, p, q, mh_iters=1,n_rows=None, debug=False):
    """
    data_path: path where data is saved.
    K: number of plants (n_plants in load_data function).
    p: past time to be considered.
    q: sample distribution for parameters.
    """

    print('Loading data...')
    Y0, X = load_data(data_path, K, p, resample_rule='10T', n_rows=n_rows)
    if debug:
        print('Y0 shape: {}'.format(Y0.shape))
        print('X shape: {}'.format(X.shape))
    
    # Theta is the vector of all parameters that will be sampled.
    # A and CovU are reshaped un a 1-D vector theta.
    theta = init_parameters(K, p, q, Y0, X, debug)
    if debug:
        print('Parameters intialized!')
    samples = []
    for i in range(iters):
        start_it = time.time()
        print('Iteration {}'.format(i))
        
        # Loop over all parameters and calculate the logLK of the parameters
        # given all others.
        for j in range(theta.shape[0]):
            #print('Theta[{}] sampling...'.format(j))
            """
            while True:
                theta[j] = q.rvs()
                lk = val_loglhood(theta, Y0, X, debug)
                if lk != -np.inf:
                    break
            """
            #print('Initialzing metropolis hastings...')
            start = time.time()
            mh_samples = metropolis_hastings(theta, j, q, mh_iters, Y0, X, debug)
            end = time.time()
            print('Time for sampling theta[{}]: {}'.format(j, end - start))
            #print('Samples obtained from metropolis:')
            #print(mh_samples)
            theta[j] = np.random.choice(mh_samples)
            #print('Random choice of the samples: {}'.format(theta[j]))
            
        print('-'*20)
        A    = np.reshape(theta[:p*K**2],(K*p,K)).swapaxes(0,1)
        CovU = np.reshape(theta[p*K**2:],(K,K)).swapaxes(0,1)
        CovU = np.dot(CovU.T,CovU)
        samples.append([A, CovU])
        end_it = time.time()
        print('Time for iteration {}: {}'.format(i, end_it - start_it))
    print('Finished!')
    return samples
        
    
def init_parameters(K, p, q, Y0, X, debug=False):
    """
    Initialization of parameters. This functions search a matrix A
    and a matrix CovU that satisfy some conditions that A and CovU
    must satisfy.
    """
    if debug:
        print('Initializing parameters...')
    while True:
        theta = np.zeros(K ** 2 * (p + 1))
        for i in range(theta.shape[0]):
            theta[i] = q.rvs()

        # Force CovU to be positive semidefinite.
        covu = np.reshape(theta[-K**2:], (K, K)).T
        covu = np.dot(covu.T, covu)
        theta[-K**2:] = np.reshape(covu, K**2)
        
        lk = val_loglhood(theta, Y0, X, debug)
        if debug:
            print('LK = {}'.format(lk))
        if lk != -np.inf:
            return theta
        

# Metropolis Hastings

In [3]:
# Metropolis Hastings

def metropolis_hastings(theta, j, q, iters, Y0, X, debug):
    """
    theta: theta vector with all parameters.
    j: theta index of the parameter currently been sampled.
    q: jumping distribution.
    """
    samples_mh = [theta[j]] # start sample.
    lk_old = val_loglhood(theta, Y0, X, debug)
    #print('Old LK: {}'.format(lk_old))
    for t in range(iters):
        lk_new = -np.inf
        #print('Sampling a parameter of theta[{}] that satisfies properties...'.format(j))
        while lk_new == -np.inf:
            x_new = q.rvs(loc=samples_mh[-1], scale=1)
            theta[j] = x_new
            lk_new = val_loglhood(theta, Y0, X, debug)
        #print('New LK: {}'.format(lk_new))
        logalpha = min([lk_new - lk_old + np.log(q.pdf(samples_mh[-1], loc=x_new) / q.pdf(x_new, loc=samples_mh[-1])), 0])
        #logalpha = min([(f(x_new, data['Time'], data['Mag'], data['Error']) - \
        #             f(samples[-1], data['Time'], data['Mag'], data['Error'])) + \
        #             np.log(q.pdf(samples[-1], mean=x_new) / q.pdf(x_new, mean=samples[-1])), 0])
        alpha = np.exp(logalpha)

        u = stats.uniform.rvs()
        if u < alpha:
            #print('Sample accepted!')
            samples_mh.append(x_new)
            lk_old = lk_new
            # accepted_mh += 1
        else:
            #print('Sample rejected! Keeping old one')
            samples_mh.append(samples_mh[-1])
        # rejected_mh += 1
        # ratio_mh.append(rejected_mh / (accepted_mh + rejected_mh))
    return np.array(samples_mh)

# Test

In [6]:
DATA_PATH = '/home/chrisams/Documents/datasets/data_TAIM/processed/'
q = stats.norm
K = 3
p = 1
iters = 2
debug = False
mh_iters = 10
n_rows = 10000 # Number of rows of the data to load

In [7]:
samples = gibbs_sampling(iters, DATA_PATH, K, p, q, mh_iters=mh_iters, n_rows=n_rows, debug=debug)

Loading data...
Iteration 0
Time for sampling theta[0]: 5.862992525100708
Time for sampling theta[1]: 5.899922609329224
Time for sampling theta[2]: 5.949949264526367


KeyboardInterrupt: 

In [ ]:
samples

In [ ]:
np.random.choice(np.array([1, 2, 3, 4]))